In [21]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd

def url_crawl(driver: webdriver.Chrome):
    url_list = []
    f = open("./linkcareer_link.txt", 'w')
    for page in range(1, 42):
        url = "https://linkareer.com/cover-letter/search?keyword=&organizationName=삼성전자&page=" + str(page) + "&tab=all"
        driver.get(url)
        try:
            driver.find_element(By.XPATH, "/html/body/div[1]/div[1]/div/div[4]/div/div[2]/div/div[3]/div[1]/div[1]/div[1]/a/div")
            driver.implicitly_wait(3)
            url_tag = driver.find_elements(By.TAG_NAME, 'a')
            for tag in url_tag:
                url_name = tag.get_attribute('href')
                if "cover-letter" in url_name and "search" not in url_name:
                    print(url_name)
                    url_list.append(url_name)
        except Exception as e:
            print(f"Error during URL crawl: {e}")
            continue
    driver.close()
    for content in list(set(url_list)):
        f.write(content + "\n")
    f.close()

def self_introduction(driver: webdriver.Chrome, url):
    person = {}
    driver.get(url)
    driver.implicitly_wait(3)
    try:
        info = driver.find_element(By.XPATH, '/html/body/div[1]/div[1]/div[4]/div/div[2]/div/div[2]/div[1]/div/div/div[2]/h1')
        specification = driver.find_element(By.XPATH, '/html/body/div[1]/div[1]/div[4]/div/div[2]/div/div[2]/div[1]/div/div/div[3]/h3')
        content = driver.find_element(By.ID, "coverLetterContent")
        info_list = info.text.split(' /')
        person['company'] = info_list[0]
        person['job'] = info_list[1]
        person['year'] = info_list[2]
        person['specification'] = specification.text  # 지원자 스펙
        person['self_intro'] = content.text  # 지원자 자소서
#         print(person)
        return person
    except Exception as e:
        print(f"Error during self introduction extraction: {url}, {e}")
        return None  # 에러가 발생하면 None 반환

# linkareer_total.py

person_list = []
url = "C:\\Users\\jenni\\Documents\\아이디어톤\\linkcareer_link.txt"
driver = webdriver.Chrome()
# url_crawl(driver=driver)
with open(url, 'r', encoding='utf-8') as f:
    while True:
        txt_link = f.readline()
        if txt_link == '':
            break
        person = self_introduction(driver=driver, url=txt_link)
        if person is not None:
            person_list.append(person)

driver.close()

df = pd.DataFrame(person_list)

# CSV 파일로 저장
df.to_csv('./self_introduction.csv', index=False, encoding='utf-8-sig')
